In [0]:
%pip install requests


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import requests, io, zipfile, json
from pyspark.sql import functions as F

# URL for the master zip
URL = "https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip"
print("Downloading zip from GitHub (~1 GB may take a few mins)...")

resp = requests.get(URL)
z = zipfile.ZipFile(io.BytesIO(resp.content))
print("✅ Zip loaded in memory with", len(z.namelist()), "files")


✅ Zip loaded in memory with 318372 files


In [0]:
import re
json_files = [f for f in z.namelist() if re.search(r"cves/2024/.+\.json$", f)]
print("Found", len(json_files), "2024 CVE files")


Found 38733 2024 CVE files


In [0]:
import datetime

flat_records = []
for rec in records:
    try:
        meta = rec.get("cveMetadata", {})
        cna = rec.get("containers", {}).get("cna", {}) if isinstance(rec.get("containers"), dict) else {}
        
        flat_records.append({
            "cveId": meta.get("cveId"),
            "state": meta.get("state"),
            "datePublished": meta.get("datePublished"),
            "dateUpdated": meta.get("dateUpdated"),
            "title": cna.get("title"),
            # Serialize complex nested data as JSON strings (for safety)
            "metrics_json": json.dumps(cna.get("metrics", [])),
            "affected_json": json.dumps(cna.get("affected", [])),
            "descriptions_json": json.dumps(cna.get("descriptions", []))
        })
    except Exception as e:
        pass  # skip malformed ones

print("✅ Flattened records:", len(flat_records))


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6116269708479902>, line 4
      1 import datetime
      3 flat_records = []
----> 4 for rec in records:
      5     try:
      6         meta = rec.get("cveMetadata", {})

NameError: name 'records' is not defined

In [0]:
df_raw = spark.createDataFrame(flat_records)
df = df_raw.withColumn("date_published_ts", F.to_timestamp("datePublished"))
df_2024 = df.filter(F.year("date_published_ts") == 2024)

print("✅ Filtered to", df_2024.count(), "records for 2024")
df_2024.printSchema()
df_2024.show(5, truncate=False)


In [0]:
# CVEs per month
df_2024.groupBy(F.date_format("date_published_ts", "yyyy-MM").alias("month")).count().orderBy("month").show(12, False)

# Count by state
df_2024.groupBy("state").count().show()

# Quick severity keyword search inside metrics JSON
df_2024.filter(F.col("metrics_json").contains("Critical")).count()
